# JUMPER STEP BY STEP

In [ ]:
import os
import pickle
import random
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.insert(0, "../")

from lib.dimensionality_reduction import dimensionality_reduction_techniques
from lib.forecast import Predictions, Simulation, load_ts
from lib.forecast_method import forecast_techniques
from lib.utils import get_dr_technique, get_forecast_technique
# import importlib
# importlib.reload(jumper)

## What parameters we can change?

### Load and Prepare simulations
- path  : path of the simulation - string
- ye    : yearly average of the simulation - bool
- start : start for the training data int
- end   : end for the training data - int
- comp  : explained variance ratio - float or int

### Predictions:
- steps : Number of steps to forecast

In [ ]:
path = "/home/ia/iccs_data/nemo_data_e1/"
ye = True

start = 0
end = 40
comp = None  # default = 0.9
steps = 10

# **LOAD & PREPARE**

#### Calling the Simulation class:
- Load sea surface height, salinity and temperature
- get other attributes (see init code)

#### Calling self.prepare function
- Cut start and end
- Compute mean, std, max etc...
- Standardize default = true

In [ ]:
# Load config file of techniques
path_to_nemo_directory = "../"
path_to_nemo_directory = Path(path_to_nemo_directory)

forecast_method = get_forecast_technique(path_to_nemo_directory, forecast_techniques)

dimensionality_reduction_method = get_dr_technique(
    path_to_nemo_directory, dimensionality_reduction_techniques
)


simu_zos = Simulation(
    path=path,
    start=start,
    end=end,
    ye=ye,
    term="ssh",
    filename="DINO_1m_To_1y_grid_T.nc",
    dimensionality_reduction=dimensionality_reduction_method,
)
print("zos loaded")  # zos=ssh
simu_so = Simulation(
    path=path,
    start=start,
    end=end,
    ye=ye,
    term="soce",
    filename="DINO_1y_grid_T.nc",
    dimensionality_reduction=dimensionality_reduction_method,
)
print("so loaded")  # so=salinity
simu_thetao = Simulation(
    path=path,
    start=start,
    end=end,
    ye=ye,
    term="toce",
    filename="DINO_1y_grid_T.nc",
    dimensionality_reduction=dimensionality_reduction_method,
)
print("thetao loaded")  # thetao=temperature


simu_zos.get_simulation_data()
print("\nzos prepared")
simu_so.get_simulation_data()
print("so prepared")
simu_thetao.get_simulation_data()
print("thetao prepared")


# LoadSimu : essayer avec des chunks plus gros et plus de jobs pour 3D
# Prepare   : Cut spin Up - Remove Closed seas - Standardize - (old : Replace bathy nan values by the mean) - to float32

In [ ]:
simus = [simu_zos, simu_so, simu_thetao]
# names = ["zos","so","thetao"]
names = ["ssh", "soce", "toce"]

fig, axes = plt.subplots(1, len(simus), figsize=(20, 4))

for i, simu in enumerate(simus):
    if simu.z_size is not None:
        im = axes[i].pcolor(simu.simulation[0, 0])
        axes[i].set_title(f"Surface {names[i]}")
    else:
        im = axes[i].pcolor(simu.simulation[0])
        axes[i].set_title(f"{names[i]}")
    plt.colorbar(im, ax=axes[i])

if False:
    fig, axes = plt.subplots(1, len(simus), figsize=(20, 4))

    for i, simu in enumerate(simus):
        if simu.z_size is not None:
            plt.plot(np.mean(simu.desc["ssca"], axis=(1, 2, 3)))
            axes[i].set_title(f"Average ssca - {names[i]}")
        else:
            plt.plot(np.mean(simu.desc["ssca"], axis=(1, 2)))
            axes[i].set_title(f"Average ssca - {names[i]}")
    plt.colorbar(im, ax=axes[i])

# **TO TIME SERIES**

#### Calling the applyPCA function:
- Compute the pca on simulation and obtain self.components and self.pca attributes

#### Calling the error function:
- Return the reconstructed simulation, rmse values and maps

#### last steps => save prepared simu and infos

In [ ]:
simu_zos.decompose()
print("PCA applied on zos")
simu_so.decompose()
print("PCA applied on so")
simu_thetao.decompose()
print("PCA applied on thetao")

In [ ]:
simus = [simu_zos, simu_so, simu_thetao]
names = ["zos", "so", "thetao"]
colors = ["tab:blue", "tab:green", "tab:red"]
fig, axes = plt.subplots(3, 3, figsize=(20, 10))

for i, simu in enumerate(simus):
    axes[0, i].plot(simu.pca.explained_variance_ratio_ * 100, "ko", markersize=4)
    axes[0, i].set_title(f"Explained Variance Ratio - {names[i]}")

    axes[1, i].plot(simu.components[:, 0], color=colors[i], alpha=0.9, label="1st comp")
    print(f"Components shape {simu.components.shape}")
    axes[1, i].plot(simu.components[:, 1], color=colors[i], alpha=0.4, label="2nd comp")
    axes[1, i].set_title(f"Components - {names[i]}")
    axes[1, i].legend()

    if simu.z_size is not None:
        im = axes[2, i].pcolor(simu.get_component(0)[0])
        plt.colorbar(im, ax=axes[2, i])  # ,label=units[i])
        axes[2, i].set_title(f"1st PC of the surface - {names[i]}")
    else:
        im = axes[2, i].pcolor(simu.get_component(0))
        plt.colorbar(im, ax=axes[2, i])  # ,label=units[i])
        axes[2, i].set_title(f"1st PC - {names[i]}")

fig.suptitle("PCA INFO")
plt.show()

In [ ]:
n = len(simu_zos.pca.explained_variance_ratio_)
rec_zos, rmseV_zos, rmseM_zos = simu_zos.error(n)
print("RMSE computed for zos")
n = len(simu_so.pca.explained_variance_ratio_)
rec_so, rmseV_so, rmseM_so = simu_so.error(n)
print("RMSE computed for so")
n = len(simu_thetao.pca.explained_variance_ratio_)
rec_thetao, rmseV_thetao, rmseM_thetao = simu_thetao.error(n)
print("RMSE computed for thetao")

In [ ]:
import xarray as xr

array = xr.open_dataset(
    simu_so.files[-1], decode_times=False, chunks={"time": 200, "x": 120}
)

In [ ]:
rmseV_zos
values = [rmseV_zos, rmseV_so.T, rmseV_thetao.T]
maps = [rmseM_zos, rmseM_so, rmseM_thetao]
names = ["zos", "so", "thetao"]
colors = ["tab:blue", "darkgreen", "darkred"]

fig = plt.figure(figsize=(6, 8))
for i in range(3):
    if i == 0:
        plt.errorbar(
            np.mean(values[i]),
            array.deptht[0],
            xerr=np.std(values[i]),
            fmt=".",
            label=names[i],
            color=colors[i],
            ecolor="grey",
        )
    else:
        plt.errorbar(
            np.mean(values[i], axis=1),
            array.deptht,
            xerr=np.std(values[i], axis=1),
            fmt=".",
            label=names[i],
            color=colors[i],
            ecolor="grey",
        )

plt.title("PCA EVALUATION - 1st COMP")
plt.ylabel("Depth")
plt.xlabel("RMSE")
plt.legend()

plt.gca().invert_yaxis()
plt.show()

In [ ]:
values = [rmseV_zos, rmseV_so, rmseV_thetao]
maps = [rmseM_zos, rmseM_so, rmseM_thetao]
names = ["zos", "so", "thetao"]

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

for i in range(3):
    if len(np.shape(maps[i])) == 2:
        im = axes[i].pcolor(maps[i])
        plt.colorbar(im, ax=axes[i])
        axes[i].set_title(f"Mean rmse map - {names[i]}")
    else:
        im = axes[i].pcolor(np.nanmean(maps[i], axis=0))
        plt.colorbar(im, ax=axes[i])
        axes[i].set_title(f"Average rmse map - {names[i]}")

fig.suptitle("PCA EVALUATION - 1st COMP")
plt.show()

In [ ]:
zos_dico = simu_zos.make_dico()
print("zos to dictionnary")
so_dico = simu_so.make_dico()
print("so to dictionnary")
thetao_dico = simu_thetao.make_dico()
print("thetao to dictionnary")

In [ ]:
# f = "/data/mtissot/spinup_data/simus_prepared/"
f = path
if not os.path.exists(f):
    os.makedirs(f)

with open(f + "pca_so", "wb") as file:
    pickle.dump(simu_so.pca, file)
with open(f + "pca_thetao", "wb") as file:
    pickle.dump(simu_thetao.pca, file)
with open(f + "pca_zos", "wb") as file:
    pickle.dump(simu_zos.pca, file)

np.savez(f + "so", **so_dico)
np.savez(f + "thetao", **thetao_dico)
np.savez(f + "zos", **zos_dico)

# **FORECAST**

#### Calling the load_ts function:
- get all components in a panda dataframe

#### Calling Predictions class:
- Get components, gp etc...

#### Calling Forecats function: 
- Compute forecast_ts for each time series

#### Calling reconstruct function: 
- reconstruct predictions and train predictions

In [ ]:
# f = "/data/mtissot/spinup_data/simus_prepared/"
f = path

df_zos, infos_zos = load_ts(f, "zos")  # TODO: Change terms? zos, so, thetao?
df_so, infos_so = load_ts(f, "so")
df_thetao, infos_thetao = load_ts(f, "thetao")

random.seed(20)

ts_zos = Predictions(
    "zos", df_zos, infos_zos, forecast_method, dimensionality_reduction_method
)
ts_so = Predictions(
    "so", df_so, infos_so, forecast_method, dimensionality_reduction_method
)
ts_thetao = Predictions(
    "thetao", df_thetao, infos_thetao, forecast_method, dimensionality_reduction_method
)

In [ ]:
import random

random.seed(100)
comp, train_len, steps = 1, len(ts_zos), steps
print(len(ts_zos))

hat_zos, std_zos, metrics_zos = ts_zos.forecast_single_series(
    comp, train_len, steps
)  # TODO: Check restart script change where I returned the simu?
hat_so, std_so, metrics_so = ts_so.forecast_single_series(comp, train_len, steps)
hat_thetao, std_thetao, metrics_thetao = ts_thetao.forecast_single_series(
    comp, train_len, steps
)

ts_zos.show(comp, hat_zos, std_zos, train_len)
ts_so.show(comp, hat_so, std_so, train_len, color="darkgreen")
ts_thetao.show(comp, hat_thetao, std_thetao, train_len, color="darkred")

In [ ]:
hat_zos, hat_std_zos, metrics = ts_zos.parallel_forecast(train_len, steps)
hat_so, hat_std_so, metrics = ts_so.parallel_forecast(train_len, steps)
hat_thetao, hat_std_thetao, metrics = ts_thetao.parallel_forecast(train_len, steps)

hat_zos = pd.concat([df_zos[:train_len], hat_zos[:]])
hat_so = pd.concat([df_so[:train_len], hat_so[:]])
hat_thetao = pd.concat([df_thetao[:train_len], hat_thetao[:]])

# print("hat zos")
# print(hat_zos)
# print(hat_zos.shape)

In [ ]:
# CHANGER PRENDRE SERIE TEMP TRUTH + PRED
n = np.shape(ts_zos.info["ts"])[1]
predictions, n, info = hat_zos, n, ts_zos.info

predictions_zos = simu_zos.reconstruct(hat_zos, n, ts_zos.info)
# TODO: Reconstruct method has been removed from class Predictions
print("zos reconstructed with all comp")

n = np.shape(ts_so.info["ts"])[1]
predictions_so = simu_so.reconstruct(hat_so, n, ts_so.info)
# TODO: Reconstruct method has been removed from class Predictions
print("so reconstructed with all comp")

n = np.shape(ts_thetao.info["ts"])[1]
predictions_thetao = simu_thetao.reconstruct(hat_thetao, n, ts_thetao.info)
# TODO: Reconstruct method has been removed from class Predictions
print("thetao reconstructed with all comp")

In [ ]:
maps = [predictions_zos, predictions_so, predictions_thetao]
names = ["zos", "so", "thetao"]

fig, axes = plt.subplots(1, len(maps), figsize=(20, 4))

for i, simu in enumerate(maps):
    if len(np.shape(simu)) > 3:
        im = axes[i].pcolor(simu[0, 0])
        axes[i].set_title(f"Surface {names[i]}")
    else:
        im = axes[i].pcolor(simu[0])
        axes[i].set_title(f"{names[i]}")
    plt.colorbar(im, ax=axes[i])

# **SAVE PREDICTIONS**

In [ ]:
if not os.path.exists(path + "simu_predicted/"):
    os.makedirs(f"{path}/simu_predicted/")

np.save(path + "simu_predicted/pred_zos.npy", predictions_zos)
np.save(path + "simu_predicted/pred_so.npy", predictions_so)
np.save(path + "simu_predicted/pred_thetao.npy", predictions_thetao)

# **EVALUATE ERROR**

In [ ]:
import sys

sys.path.insert(0, "../lib/")

# import forecast
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from lib.forecast import Predictions, Simulation, load_ts
# import importlib
# importlib.reload(p1)

In [ ]:
pred_zos = np.load(path + "simu_predicted/pred_zos.npy")
pred_so = np.load(path + "simu_predicted/pred_so.npy")
pred_thetao = np.load(path + "simu_predicted/pred_thetao.npy")

In [ ]:
id_, start2, end2 = "106", start, end + steps  # start,end+steps


ye = False

ref_zos = Simulation(
    path=path,
    start=start2,
    end=end2,
    ye=ye,
    term="ssh",
    filename="DINO_1m_To_1y_grid_T.nc",
    dimensionality_reduction=dimensionality_reduction_method,
)
print("zos loaded")
ref_so = Simulation(
    path=path,
    start=start2,
    end=end2,
    ye=ye,
    term="soce",
    filename="DINO_1y_grid_T.nc",
    dimensionality_reduction=dimensionality_reduction_method,
)
print("so loaded")
ref_thetao = Simulation(
    path=path,
    start=start2,
    end=end2,
    ye=ye,
    term="toce",
    filename="DINO_1y_grid_T.nc",
    dimensionality_reduction=dimensionality_reduction_method,
)
print("thetao loaded")

# REMTTRE BIEN GET DATA APRES      return grid[:1] => return grid
ref_zos.get_simulation_data(stand=False)
print("\nzos prepared")
ref_so.get_simulation_data(stand=False)
print("so prepared")
ref_thetao.get_simulation_data(stand=False)
print("thetao prepared")

In [ ]:
array = xr.open_dataset(
    ref_so.files[-1], decode_times=False, chunks={"time": 200, "x": 120}
)
depth = array.deptht
del array

#### ERREUR PREDICTIONS

In [ ]:
print(ref_so.simulation[end:].shape)

err_so = np.abs((ref_so.simulation - pred_so))
err_thetao = np.abs((ref_thetao.simulation - pred_thetao))
err_zos = np.abs((ref_zos.simulation - pred_zos))

# predictions
i = steps
mean_err_so_pred = np.nanmean(err_so[-i:], axis=(0, 2, 3))
std_err_so_pred = np.nanstd(err_so[-i:], axis=(0, 2, 3))

mean_err_thetao_pred = np.nanmean(err_thetao[-i:], axis=(0, 2, 3))
std_err_thetao_pred = np.nanstd(err_thetao[-i:], axis=(0, 2, 3))

mean_err_zos_pred = np.nanmean(err_zos[-i:], axis=(0, 1, 2))
std_err_zos_pred = np.nanstd(err_zos[-i:], axis=(0, 1, 2))

# reference
i = end
mean_err_so_ref = np.nanmean(err_so[:-i], axis=(0, 2, 3))
std_err_thetao_ref = np.nanstd(err_so[:-i], axis=(0, 2, 3))

mean_err_thetao_ref = np.nanmean(err_thetao[:-i], axis=(0, 2, 3))
std_err_so_ref = np.nanstd(err_thetao[:-i], axis=(0, 2, 3))

mean_err_zos_ref = np.nanmean(err_zos[:-i], axis=(0, 1, 2))
std_err_zos_ref = np.nanstd(err_zos[:-i], axis=(0, 1, 2))

In [ ]:
categories = ["Prediction", "PCA"]
means = [mean_err_zos_pred, mean_err_zos_ref]
print(means)
errors = [std_err_zos_pred, std_err_zos_ref]

fig, ax = plt.subplots(figsize=(6, 3))
ax.bar(categories, means, yerr=errors, capsize=5, color=["tab:blue", "grey"])

ax.set_title("Mean Error with Standard Error")
ax.set_ylabel("Error")
ax.set_xlabel("Categories")
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 6))

# Plot thetao infos
axes[0].plot(
    depth,
    mean_err_thetao_ref,
    color="black",
    label="thetao",
    linestyle="dashed",
    alpha=0.6,
)
axes[0].fill_between(
    depth,
    mean_err_thetao_ref + std_err_thetao_ref,
    mean_err_thetao_ref - std_err_thetao_ref,
    color="black",
    alpha=0.1,
)

axes[0].plot(depth, mean_err_thetao_pred, color="darkred", label="thetao")
axes[0].fill_between(
    depth,
    mean_err_thetao_pred + std_err_thetao_pred,
    mean_err_thetao_pred - std_err_thetao_pred,
    color="darkred",
    alpha=0.2,
)
axes[0].set_xlabel("Depth")
axes[0].set_ylabel("Mean Error")
axes[0].legend()

# Plot so infos
axes[1].plot(
    depth, mean_err_so_ref, color="black", label="so", linestyle="dashed", alpha=0.6
)
axes[1].fill_between(
    depth,
    mean_err_so_ref + std_err_so_ref,
    mean_err_so_ref - std_err_so_ref,
    color="black",
    alpha=0.1,
)

axes[1].plot(depth, mean_err_so_pred, color="darkgreen", label="so")
axes[1].fill_between(
    depth,
    mean_err_so_pred + std_err_so_pred,
    mean_err_so_pred - std_err_so_pred,
    color="darkgreen",
    alpha=0.2,
)
axes[1].set_xlabel("Depth")
axes[1].set_ylabel("Mean Error")
axes[1].legend()

fig.suptitle(f"Absolute error on {i} last predictions")
plt.tight_layout()
plt.show()

In [ ]:
mean_pred_so = np.nanmean(
    pred_so, axis=(0, 2, 3)
)  # TODO: Ensure ref_so only takes the time interval corresponding to the predictions
mean_ref_so = np.nanmean(ref_so.simulation, axis=(0, 2, 3))
print("pred_so shape", pred_so.shape)
print("ref_so shape", ref_so.simulation.shape)
print("mean_pred_so shape", mean_pred_so.shape)
print("mean_ref_so shape", mean_ref_so.shape)
mean_pred_thetao = np.nanmean(pred_thetao, axis=(0, 2, 3))
mean_ref_thetao = np.nanmean(ref_thetao.simulation, axis=(0, 2, 3))
print("pred_thetao shape", pred_thetao.shape)
print("ref_thetao shape", ref_thetao.simulation.shape)
print("mean_pred_thetao shape", mean_pred_thetao.shape)
print("mean_ref_thetao shape", mean_ref_thetao.shape)
mean_pred_zos = np.nanmean(pred_zos, axis=(1, 2))
mean_ref_zos = np.nanmean(ref_zos.simulation, axis=(1, 2))
print("pred_zos shape", pred_zos.shape)
print("ref_zos shape", ref_zos.simulation.shape)
print("mean_pred_zos shape", mean_pred_zos.shape)
print("mean_ref_zos shape", mean_ref_zos.shape)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
print("depth shape: ", depth.shape)
print("mean_pred_so shape: ", mean_pred_so.shape)
axes[0].plot(depth, mean_pred_so, label="predictions")
axes[0].plot(depth, mean_ref_so, label="reference")
axes[0].set_title("Salinity")
axes[0].legend()

axes[1].plot(depth, mean_pred_thetao, label="predictions")
axes[1].plot(depth, mean_ref_thetao, label="reference")
axes[1].set_title("Temperature")
axes[1].legend()

fig.suptitle("Average over depth")
plt.tight_layout()
plt.show()

In [ ]:
mean_pred = [mean_pred_zos, mean_pred_so, mean_pred_thetao]
mean_ref = [mean_ref_zos, mean_ref_so, mean_ref_thetao]
names = ["zos", "so", "thetao"]
colors = ["tab:blue", "darkgreen", "darkred"]

fig, axes = plt.subplots(3, 1, figsize=(10, 8))
for i, ax in enumerate(axes):
    ax.plot(mean_pred[i], color=colors[i], label=names[i])
    ax.plot(mean_ref[i], color="grey", label="ref", linestyle="dashed")
    ax.legend()

### Evaluate time series Predictions

In [ ]:
ref_zos.simulation = (ref_zos.simulation - ref_zos.desc["mean"]) / (
    2 * ref_zos.desc["std"]
)
ref_so.simulation = (ref_so.simulation - ref_so.desc["mean"]) / (2 * ref_so.desc["std"])
ref_thetao.simulation = (ref_thetao.simulation - ref_thetao.desc["mean"]) / (
    2 * ref_thetao.desc["std"]
)

ref_zos.decompose()
print("PCA applied on zos")
ref_so.decompose()
print("PCA applied on so")
ref_thetao.decompose()
print("PCA applied on thetao")

In [ ]:
simus = [ref_zos, ref_so, ref_thetao]
names = ["zos", "so", "thetao"]
colors = ["tab:blue", "tab:green", "tab:red"]
fig, axes = plt.subplots(3, 3, figsize=(20, 10))

for i, simu in enumerate(simus):
    axes[0, i].plot(simu.pca.explained_variance_ratio_ * 100, "ko", markersize=4)
    axes[0, i].set_title(f"Explained Variance Ratio - {names[i]}")

    axes[1, i].plot(simu.components[:, 0], color=colors[i], alpha=0.9, label="1st comp")
    axes[1, i].plot(simu.components[:, 1], color=colors[i], alpha=0.4, label="2nd comp")
    axes[1, i].set_title(f"Components - {names[i]}")
    axes[1, i].legend()

    if simu.z_size is not None:
        im = axes[2, i].pcolor(simu.get_component(0)[0])
        plt.colorbar(im, ax=axes[2, i])  # ,label=units[i])
        axes[2, i].set_title(f"1st PC of the surface - {names[i]}")
    else:
        im = axes[2, i].pcolor(simu.get_component(0))
        plt.colorbar(im, ax=axes[2, i])  # ,label=units[i])
        axes[2, i].set_title(f"1st PC - {names[i]}")

fig.suptitle("PCA INFO")
plt.show()

In [ ]:
comp = 0
ref = [ref_zos, ref_so, ref_thetao]
pred = [hat_zos, hat_so, hat_thetao]  # [df_zos,df_so,df_thetao]
names = ["zos", "so", "thetao"]
colors = ["tab:blue", "tab:green", "tab:red"]
total_len = len(ref[0].simulation)
fig, axes = plt.subplots(3, 1, figsize=(10, 8))

for i, simu in enumerate(ref):
    axes[i].plot(
        simu.components[:, comp], color="grey", linestyle="dashed", label="ref"
    )
    axes[i].plot(
        np.arange(0, total_len),
        pred[i].iloc[:, comp],
        color=colors[i],
        alpha=0.9,
        label=names[i],
    )
    axes[i].set_title(f"Components - {names[i]}")
    axes[i].legend()

fig.suptitle("PCA INFO")
plt.show()

In [ ]:
comp = 1

fig, axes = plt.subplots(3, 1, figsize=(10, 8))

for i, simu in enumerate(ref):
    axes[i].plot(
        simu.components[:, comp], color="grey", linestyle="dashed", label="ref"
    )
    axes[i].plot(
        np.arange(0, total_len),
        pred[i].iloc[:, comp],
        color=colors[i],
        alpha=0.9,
        label=names[i],
    )
    axes[i].set_title(f"Components - {names[i]}")
    axes[i].legend()

fig.suptitle("PCA INFO")
plt.show()

In [ ]:
comp = 2

fig, axes = plt.subplots(3, 1, figsize=(10, 8))

for i, simu in enumerate(ref):
    axes[i].plot(
        simu.components[:, comp], color="grey", linestyle="dashed", label="ref"
    )
    axes[i].plot(
        np.arange(0, total_len),
        pred[i].iloc[:, comp],
        color=colors[i],
        alpha=0.9,
        label=names[i],
    )
    # axes[i].set_xticks(np.arange(0, total_len))
    axes[i].set_title(f"Components - {names[i]}")
    axes[i].legend()

fig.suptitle("PCA INFO")
plt.show()